In [1]:
%pip install lyricsgenius
%pip install spotipy
%pip install spotipy lyricsgenius transformers scikit-learn gtts pydub librosa
%pip install faiss-cpu
%pip install tqdm

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.7-py3-none-any.whl.metadata (9.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 7.9 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 6.8 MB/s eta 0:00:00
Using cached idna-3.7-py3-none-any.whl (66 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 6.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.6/255.6 kB 8.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
     ━━━━━━━━━━━━

In [2]:
# import pandas as pd


# data = pd.read_csv("spotify/data/data.csv")
# genre_data = pd.read_csv('spotify/data/data_by_genres.csv')
# year_data = pd.read_csv('spotify/data/data_by_year.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'spotify/data/data.csv'

In [ ]:
# import os
# import pandas as pd
# import tqdm 

# # show stahe of the progress bar
# tqdm.tqdm.pandas()

# # Setting the base directory using list of directory names
# base_dir = "data"

# # Building paths by further extending the base directory
# data_path = os.path.join(base_dir, "data.csv")
# genre_data_path = os.path.join(base_dir, "data_by_genres.csv")
# year_data_path = os.path.join(base_dir, "data_by_year.csv")

# # Reading the data using pandas
# data = pd.read_csv(data_path)
# genre_data = pd.read_csv(genre_data_path)
# year_data = pd.read_csv(year_data_path)


In [ ]:
# # spotify 


# import spotipy
# client_id = '10cc8ee290404da9ab9d7b061d526193'
# client_secret = '0dc9cb56d8bc4454afa1ddbe82a7301d'

# tqdm.tqdm.pandas()

# from spotipy.oauth2 import SpotifyClientCredentials
# client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# # check health 
# sp.trace = False
# track = sp.track('7qiZfU4dY1lWllzX7mPBI3')
# print(track['name'])
# # Now you have the access token to make requests to the Spotify API


In [ ]:
# # login to spotfy account as user and get their playlists
# import spotipy.util as util

# username = 'eqanbww3jh63cgf4ot5zyyr5d'
# scope = 'playlist-read-private'
# token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri='http://localhost:8888/callback')
# if token:
#     sp = spotipy.Spotify(auth=token)
#     playlists = sp.user_playlists(username)
#     for playlist in playlists['items']:
#         print(playlist['name'])

# else:
#     print("Can't get token for", username)

# # list songs in all the playlist 


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import lyricsgenius
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import faiss


# Replace these with your own Spotify and Genius credentials
SPOTIFY_CLIENT_ID = '***REMOVED***'
SPOTIFY_CLIENT_SECRET = '***REMOVED***'
SPOTIFY_REDIRECT_URI = 'http://localhost:8888/callback'
GENIUS_API_TOKEN = '***REMOVED***'

# Spotify API setup
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIFY_CLIENT_ID,
                                               client_secret=SPOTIFY_CLIENT_SECRET,
                                               redirect_uri=SPOTIFY_REDIRECT_URI,
                                               scope="user-top-read user-library-read playlist-read-private"))

# Genius API setup
genius = lyricsgenius.Genius(GENIUS_API_TOKEN)

# Load pre-trained transformer model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

# Force model to use CPU to avoid GPU memory issues
device = torch.device("cpu")

def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()

def get_spotify_top_tracks(sp, limit=5, time_range='medium_term'):
    print("Fetching top tracks from Spotify...")
    results = sp.current_user_top_tracks(limit=limit, time_range=time_range)
    tracks = results['items']
    return tracks

def get_spotify_playlists(sp):
    print("Fetching playlists from Spotify...")
    results = sp.current_user_playlists()
    playlists = results['items']
    return playlists

def get_audio_features(sp, track_ids):
    print("Fetching audio features from Spotify...")
    audio_features = sp.audio_features(track_ids)
    return audio_features

def get_lyrics(artist, title):
    song = genius.search_song(title, artist)
    if song:
        return song.lyrics
    return None

def audio_features_to_vector(audio_features):
    print(audio_features)
    vector = np.array([
        audio_features['danceability'],
        audio_features['energy'],
        audio_features['speechiness'],
        audio_features['acousticness'],
        audio_features['instrumentalness'],
        audio_features['liveness'],
        audio_features['valence'],
        audio_features['tempo']
    ])
    return vector

def create_faiss_index(data, dimension):
    print("Creating FAISS index...")
    index = faiss.IndexFlatL2(dimension)
    index.add(data)
    return index

# Fetch and process data
def fetch_and_process_data(sp, limit=5):
    print("Fetching data from Spotify...")
    tracks = get_spotify_top_tracks(sp, limit=limit)
    track_ids = [track['id'] for track in tracks]
    audio_features = get_audio_features(sp, track_ids)
    
    playlists = get_spotify_playlists(sp)
    playlist_names = [playlist['name'] for playlist in playlists]
    
    lyrics_data = []
    audio_vectors = []
    
    for track, audio_feature in zip(tracks, audio_features):
        artist = track['artists'][0]['name']
        title = track['name']
        lyrics = get_lyrics(artist, title)
        if lyrics:
            lyrics_embedding = embed_text(lyrics)
            audio_vector = audio_features_to_vector(audio_feature)
            lyrics_data.append({'id': track['id'], 'embedding': lyrics_embedding, 'artist': artist, 'title': title, 'lyrics': lyrics})
            audio_vectors.append(audio_vector)
    
    lyrics_embeddings = np.vstack([song['embedding'] for song in lyrics_data])
    audio_vectors = np.vstack(audio_vectors)
    
    return lyrics_data, lyrics_embeddings, audio_vectors, playlist_names

lyrics_data, lyrics_embeddings, audio_vectors, playlist_names = fetch_and_process_data(sp, limit=5)





In [ ]:
# Create FAISS indices
lyrics_index = create_faiss_index(lyrics_embeddings, 768)
audio_index = create_faiss_index(audio_vectors, 8)

def get_user_mood(input_text):
    print("Processing user input...")       
    input_embedding = embed_text(input_text)
    lyrics_distances, lyrics_indices = lyrics_index.search(input_embedding, k=5)
    audio_distances, audio_indices = audio_index.search(audio_vectors, k=5)

    combined_results = {}
    for i, index in enumerate(lyrics_indices[0]):
        combined_results[index] = combined_results.get(index, 0) + (1 / (lyrics_distances[0][i] + 1e-6))
    for i, index in enumerate(audio_indices[0]):
        combined_results[index] = combined_results.get(index, 0) + (1 / (audio_distances[0][i] + 1e-6))

    sorted_results = sorted(combined_results.items(), key=lambda x: x[1], reverse=True)
    return [lyrics_data[index] for index, _ in sorted_results]


In [3]:
%pip install torch

  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 7.8 MB/s eta 0:00:0000:0100:01
Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 3.7 MB/s eta 0:00:0000:0100:01
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import lyricsgenius
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import faiss
import gc  # For garbage collection
import logging
import psutil  # For memory monitoring

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Replace these with your own Spotify and Genius credentials
SPOTIFY_CLIENT_ID = '***REMOVED***'
SPOTIFY_CLIENT_SECRET = '***REMOVED***'
SPOTIFY_REDIRECT_URI = 'http://localhost:8888/callback'
GENIUS_API_TOKEN = '***REMOVED***'

# Spotify API setup
logger.info("Setting up Spotify API...")
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIFY_CLIENT_ID,
                                               client_secret=SPOTIFY_CLIENT_SECRET,
                                               redirect_uri=SPOTIFY_REDIRECT_URI,
                                               scope="user-top-read user-library-read playlist-read-private"))

# Genius API setup
logger.info("Setting up Genius API...")
genius = lyricsgenius.Genius(GENIUS_API_TOKEN)

# Load pre-trained transformer model
logger.info("Loading pre-trained transformer model...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

# Force model to use CPU to avoid GPU memory issues
device = torch.device("cpu")

def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()

def log_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    logger.info(f"Memory usage: {mem_info.rss / 1024 ** 2:.2f} MB")

def get_spotify_top_tracks(sp, limit=5, time_range='medium_term'):
    logger.info(f"Fetching top {limit} tracks from Spotify...")
    results = sp.current_user_top_tracks(limit=limit, time_range=time_range)
    tracks = results['items']
    logger.info(f"Fetched {len(tracks)} tracks.")
    log_memory_usage()
    return tracks

def get_spotify_playlists(sp):
    logger.info("Fetching user playlists from Spotify...")
    results = sp.current_user_playlists()
    playlists = results['items']
    logger.info(f"Fetched {len(playlists)} playlists.")
    log_memory_usage()
    return playlists

def get_audio_features(sp, track_ids):
    logger.info("Fetching audio features from Spotify...")
    audio_features = sp.audio_features(track_ids)
    logger.info(f"Fetched audio features for {len(audio_features)} tracks.")
    log_memory_usage()
    return audio_features

def get_lyrics(artist, title):
    logger.info(f"Fetching lyrics for {title} by {artist} from Genius...")
    song = genius.search_song(title, artist)
    if song:
        logger.info(f"Fetched lyrics for {title}.")
        return song.lyrics
    logger.warning(f"Lyrics for {title} by {artist} not found.")
    return None

def audio_features_to_vector(audio_features):
    vector = np.array([
        audio_features['danceability'],
        audio_features['energy'],
        audio_features['speechiness'],
        audio_features['acousticness'],
        audio_features['instrumentalness'],
        audio_features['liveness'],
        audio_features['valence'],
        audio_features['tempo']
    ])
    return vector

def create_faiss_index(data, dimension):
    logger.info(f"Creating FAISS index with dimension {dimension}...")
    index = faiss.IndexFlatL2(dimension)
    index.add(data)
    logger.info("FAISS index created.")
    log_memory_usage()
    return index

# Fetch and process data in batches
def fetch_and_process_data(sp, limit=5, batch_size=2):
    tracks = get_spotify_top_tracks(sp, limit=limit)
    track_ids = [track['id'] for track in tracks]
    audio_features = get_audio_features(sp, track_ids)
    
    playlists = get_spotify_playlists(sp)
    playlist_names = [playlist['name'] for playlist in playlists]
    
    lyrics_data = []
    audio_vectors = []
    
    for i in range(0, len(tracks), batch_size):
        batch_tracks = tracks[i:i+batch_size]
        batch_audio_features = audio_features[i:i+batch_size]
        
        for track, audio_feature in zip(batch_tracks, batch_audio_features):
            artist = track['artists'][0]['name']
            title = track['name']
            lyrics = get_lyrics(artist, title)
            if lyrics:
                lyrics_embedding = embed_text(lyrics)
                audio_vector = audio_features_to_vector(audio_feature)
                lyrics_data.append({'id': track['id'], 'embedding': lyrics_embedding, 'artist': artist, 'title': title, 'lyrics': lyrics})
                audio_vectors.append(audio_vector)
        
        # Explicitly release memory
        del batch_tracks, batch_audio_features
        gc.collect()
        log_memory_usage()
    
    lyrics_embeddings = np.vstack([song['embedding'] for song in lyrics_data])
    audio_vectors = np.vstack(audio_vectors)
    
    return lyrics_data, lyrics_embeddings, audio_vectors, playlist_names

logger.info("Fetching and processing data...")
lyrics_data, lyrics_embeddings, audio_vectors, playlist_names = fetch_and_process_data(sp, limit=5)

# Create FAISS indices
lyrics_index = create_faiss_index(lyrics_embeddings, 768)
audio_index = create_faiss_index(audio_vectors, 8)

def get_user_mood(input_text):
    try:
        logger.info("Embedding user input text...")
        input_embedding = embed_text(input_text)
        logger.info("Searching for similar lyrics...")
        lyrics_distances, lyrics_indices = lyrics_index.search(input_embedding, k=5)
        logger.info("Searching for similar audio features...")
        audio_distances, audio_indices = audio_index.search(input_embedding, k=5)  # Changed to input_embedding

        combined_results = {}
        for i, index in enumerate(lyrics_indices[0]):
            combined_results[index] = combined_results.get(index, 0) + (1 / (lyrics_distances[0][i] + 1e-6))
        for i, index in enumerate(audio_indices[0]):
            combined_results[index] = combined_results.get(index, 0) + (1 / (audio_distances[0][i] + 1e-6))

        sorted_results = sorted(combined_results.items(), key=lambda x: x[1], reverse=True)
        logger.info("Recommendations generated.")
        log_memory_usage()
        return [lyrics_data[index] for index, _ in sorted_results]
    except Exception as e:
        logger.error(f"An error occurred: {e}")
        logger.error(e.with_traceback)
        raise

# Example user input
user_input = "I feel really happy and energetic today"
logger.info("Generating recommendations...")
recommendations = get_user_mood(user_input)

logger.info("Recommended songs:")
for song in recommendations:
    logger.info(f" - {song['title']} by {song['artist']}")

logger.info("Your Playlist Names:")
for name in playlist_names:
    logger.info(f" - {name}")


INFO:__main__:Setting up Spotify API...
INFO:__main__:Setting up Genius API...
INFO:__main__:Loading pre-trained transformer model...


ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
